# Install VectorVault

In [1]:
!pip install vector-vault 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.8 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=584002 sha256=061553e6cec12c83c43872e6d8cba30e2ba3ecff68657d3858bbf07f319202ca
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243

<br>
<br>

## Get API key to access the Vault Cloud

Enter your name, email, and make a password below to get an API key.

In [ ]:
from vectorvault import register

register(first_name='John', last_name='Smith', email='john@smith.com', password='make_a_password')

<br>

# Connect to VectorVault
In the following code, we import the main Vault class to connect to a vault as well as set our OpenAI API key as an environment varible. We will need that set to process the vectors with `get_vectors` and use the `get_chat()` function later.

In [ ]:
%env OPENAI_API_KEY = YOUR_OPENAI_API_KEY

In [9]:
from vectorvault import Vault

vault = Vault(user='your@email.com', api_key='YOUR_VECTOR_VAULT_API_KEY', vault='philosophy', verbose=True)

Connected to Vault: philosophy


<br>

# Download the Philosophy Books 
1. Download the book texts from the handpicked, refinded data repo of no-copywrite philosophy texts
2. Get vectors 
3. Add to Vault

This is a large amount of text and will take roughly 9min to get the vectors and then save them to the cloud. After this is done, referencing the all of them at once will take less than a second.

*A Free account will not have the resources to fully finish this. You can stop after the first couple books, or get a [Personal account](https://buy.stripe.com/7sI03AgMldS5b7y7st)*

In [10]:
import re
import requests 
from vectorvault import download_url

# Get the books from the repo
response = requests.get("https://api.github.com/repos/John-Rood/Philosophy/contents/books")
files = response.json()

# Make a loop to download each book since there are 37
for file in files:
  if file["name"].endswith(".txt"):

    # Download the book text
    file_url = file["download_url"]
    text = download_url(file_url)
    print(f"Processing {file_url}")

    # Extract metadata
    title = re.search(r"Title:\s*(.+)", text)
    author = re.search(r"Author:\s*(.+)", text)

    # Handle formatting issues
    title = title.group(1).strip() if title else "Unknown Title"
    author = author.group(1).strip() if author else "Unknown Author"

    meta = {
              "title": title,
              "author": author,
            }

    # Add for processing
    vault.add(text, meta)

    # Get vectors for all data added
    vault.get_vectors()
    
    # Save data with vectors to the Vault Cloud
    vault.save()


Deleting started. Note: this can take a while for large datasets
Vault deleted
Processing https://raw.githubusercontent.com/John-Rood/Philosophy/main/books/A%20Treatise%20of%20Human%20Nature%20-%20David%20Hume.txt
Text length too long. Using the built-in "split_text()" function to get a list of text segments
initialize index --- 3.501586675643921 seconds ---
get vectors time --- 5.763747453689575 seconds ---
save vectors time --- 118.0846655368805 seconds ---
Processing https://raw.githubusercontent.com/John-Rood/Philosophy/main/books/Aesop%E2%80%99s%20Fables%20-%20Aesop.txt
Text length too long. Using the built-in "split_text()" function to get a list of text segments
get load vectors --- 0.17588448524475098 seconds ---
initialize index --- 0.34125566482543945 seconds ---
Projected Tokens per min:27072.20043459577 | Rate Limit Ratio: 0.07734914409884505 | Text Length: 56133
Time calc'd to sleep: 0
get vectors time --- 1.2651512622833252 seconds ---
save vectors time --- 23.54102468490

# Ask Your Question

In [13]:
from vectorvault import wrap

user_input = input("What's your next question?: ")

# Get response from Language model
answer = vault.get_chat(user_input, get_context=True)

print("Question:", f"\n{wrap(user_input)}", "\n\nAnswer:", f"\n{wrap(answer)}")


What's your next question?: What is the meaning of life?
Projected Tokens per min:11.16041032498934 | Rate Limit Ratio: 0.0001240045591665482 | Text Length: 7
Time calc'd to sleep: 0
get chat time --- 10.88690185546875 seconds ---
Question: 
What is the meaning of life? 

Answer: 
The meaning of life is a question that has been pondered by philosophers,
theologians, and individuals throughout history. However, it is not a question
that has a universal answer that applies to everyone. Each person must find
their own meaning and purpose in life through their experiences, beliefs, and
values. It is important to reflect on what truly brings us joy, fulfillment, and
a sense of purpose, and to strive towards those things. Whether it is through
personal growth, helping others, or pursuing our passions, the meaning of life
is ultimately what we make of it.


<br>

## See where the answer is coming from with `return_context=True`
In this response, we add conversation history to to the "get_chat()" function. We also print the source context that the llm uses for the response. (Since the sources are returned along with the response, it comes in the form of a dictionary)

In [16]:
user_input2 = input("What's your next question?: ")

history = user_input + answer

# Get response from Language model
vault_response = vault.get_chat(user_input2, history=history, get_context=True, return_context=True)

# Since 'return_context=True', the response is returned in a dictionary
answer = vault_response['response']
print("Question:", f"\n{wrap(user_input2)}", "\n\nAnswer:", f"\n{wrap(answer)}")

# show the context used to generate the answer
for item in vault_response['context']:
    print("\n\n", f"Title: {item['metadata']['title']}")
    print(f"Author: {item['metadata']['author']}","\n\n")
    print(wrap(item['data']))


What's your next question?: How should I live each day?
Projected Tokens per min:6.149171957182388 | Rate Limit Ratio: 6.832413285758209e-05 | Text Length: 7
Time calc'd to sleep: 0
get chat time --- 14.963200330734253 seconds ---
Question: 
How should I live each day? 

Answer: 
Live each day with intention and purpose. Consider what brings you joy and
fulfillment, and make an effort to incorporate those things into your daily
life. It's important to practice gratitude and kindness towards others, as well
as to embrace opportunities for personal growth and self-improvement. Remember
that life is a gift, and each moment is precious. Focus on living in the present
and making the most of each day, and you will find meaning and purpose in your
life. As philosophers such as Antoninus have taught us, there may be
difficulties and suffering in the world, but we must focus on what is within our
control and not let external events dictate our happiness. By living a life of
purpose, gratitude, 

In [17]:
user_input3 = input("What's your next question?: ")

history = history + user_input2 + answer

# Get response from Language model
response = vault.get_chat(user_input3, history=history, get_context=True)

print("Question:", f"\n{wrap(user_input3)}", "\n\nAnswer:", f"\n{wrap(response)}")


What's your next question?: How can I make more money?
Projected Tokens per min:2.90621538815631 | Rate Limit Ratio: 3.2291282090625665e-05 | Text Length: 7
Time calc'd to sleep: 0
get chat time --- 11.208943605422974 seconds ---
Question: 
How can I make more money? 

Answer: 
Unfortunately, this context does not provide information on how to make more
money. However, it does offer guidance on living a fulfilling life through
intention, gratitude, and personal growth. Perhaps by focusing on these values,
we can find ways to improve our financial situation in a way that aligns with
our true purpose and values. Additionally, it may be useful to seek out
resources and guidance from experts in finance or entrepreneurship to help us
reach our financial goals. Remember, living a meaningful life involves more than
just money, but financial stability can certainly contribute to our overall
well-being.
